In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Bitcoin Keys and Addresses\n",
    "\n",
    "This notebook demonstrates how to use the `bitcoinutils` library to work with Bitcoin keys and addresses. We'll cover:\n",
    "\n",
    "1. Creating and using private keys\n",
    "2. Deriving public keys\n",
    "3. Creating different types of addresses\n",
    "4. Signing and verifying messages"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Setup\n",
    "\n",
    "First, let's import the necessary modules and set up the network."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from bitcoinutils.setup import setup\n",
    "from bitcoinutils.keys import PrivateKey, PublicKey, P2pkhAddress, P2shAddress, P2wpkhAddress, P2wshAddress, P2trAddress\n",
    "from bitcoinutils.script import Script\n",
    "\n",
    "# Always remember to setup the network\n",
    "# For real Bitcoin, use 'mainnet'\n",
    "setup('testnet')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Working with Private Keys\n",
    "\n",
    "Private keys are the foundation of Bitcoin ownership. Let's create and manipulate private keys."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate a random private key\n",
    "random_priv_key = PrivateKey()\n",
    "print(f\"Random private key (WIF): {random_priv_key.to_wif()}\")\n",
    "\n",
    "# Create a private key from a known WIF\n",
    "known_wif = \"cVdte9ei2xsVjmZSPtyucG43YZgNkmKTqhwiUA8M4Fc3W8Xv9cmu\"  # Example testnet WIF\n",
    "priv_key_from_wif = PrivateKey.from_wif(known_wif)\n",
    "print(f\"Private key from WIF: {priv_key_from_wif.to_wif()}\")\n",
    "\n",
    "# Create a private key from bytes\n",
    "import os\n",
    "random_bytes = os.urandom(32)  # 32 random bytes\n",
    "priv_key_from_bytes = PrivateKey.from_bytes(random_bytes)\n",
    "print(f\"Private key from bytes (WIF): {priv_key_from_bytes.to_wif()}\")\n",
    "\n",
    "# Create a private key from a secret exponent (an integer)\n",
    "secret_exponent = 12345  # Very insecure, for demonstration only!\n",
    "priv_key_from_int = PrivateKey(secret_exponent=secret_exponent)\n",
    "print(f\"Private key from secret exponent (WIF): {priv_key_from_int.to_wif()}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Deriving Public Keys\n",
    "\n",
    "Public keys are derived from private keys using elliptic curve cryptography."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate a private key\n",
    "priv_key = PrivateKey()\n",
    "print(f\"Private key (WIF): {priv_key.to_wif()}\")\n",
    "\n",
    "# Derive the corresponding public key\n",
    "pub_key = priv_key.get_public_key()\n",
    "print(f\"Public key (compressed): {pub_key.to_hex()}\")\n",
    "print(f\"Public key (uncompressed): {pub_key.to_hex(compressed=False)}\")\n",
    "\n",
    "# For Taproot, we need the x-only coordinate\n",
    "x_only_pubkey = pub_key.to_x_only_hex()\n",
    "print(f\"X-only public key: {x_only_pubkey}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Creating Different Types of Addresses\n",
    "\n",
    "Bitcoin supports various address formats. Let's create addresses for different use cases."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate a private key\n",
    "priv_key = PrivateKey()\n",
    "pub_key = priv_key.get_public_key()\n",
    "\n",
    "# Create a legacy P2PKH address\n",
    "p2pkh_addr = pub_key.get_address()\n",
    "print(f\"P2PKH address: {p2pkh_addr.to_string()}\")\n",
    "\n",
    "# Create a SegWit v0 P2WPKH address\n",
    "p2wpkh_addr = pub_key.get_segwit_address()\n",
    "print(f\"P2WPKH address: {p2wpkh_addr.to_string()}\")\n",
    "\n",
    "# Create a P2SH-P2WPKH address (nested SegWit)\n",
    "p2sh_p2wpkh_addr = pub_key.get_p2sh_p2wpkh_address()\n",
    "print(f\"P2SH-P2WPKH address: {p2sh_p2wpkh_addr.to_string()}\")\n",
    "\n",
    "# Create a Taproot (SegWit v1) address\n",
    "p2tr_addr = pub_key.get_taproot_address()\n",
    "print(f\"P2TR address: {p2tr_addr.to_string()}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Creating Addresses from Scripts\n",
    "\n",
    "Bitcoin allows creating addresses from scripts, which enables more complex spending conditions."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create two public keys for a multisig script\n",
    "priv_key1 = PrivateKey()\n",
    "priv_key2 = PrivateKey()\n",
    "pub_key1 = priv_key1.get_public_key()\n",
    "pub_key2 = priv_key2.get_public_key()\n",
    "\n",
    "# Create a 2-of-2 multisig script\n",
    "multisig_script = Script([2, pub_key1.to_hex(), pub_key2.to_hex(), 2, 'OP_CHECKMULTISIG'])\n",
    "print(f\"Multisig script: {multisig_script.to_string()}\")\n",
    "\n",
    "# Create a P2SH address from the multisig script\n",
    "p2sh_addr = P2shAddress.from_script(multisig_script)\n",
    "print(f\"P2SH address: {p2sh_addr.to_string()}\")\n",
    "\n",
    "# Create a P2WSH address from the multisig script\n",
    "p2wsh_addr = multisig_script.get_segwit_address()\n",
    "print(f\"P2WSH address: {p2wsh_addr.to_string()}\")\n",
    "\n",
    "# Create a P2SH-P2WSH address from the multisig script\n",
    "p2sh_p2wsh_addr = multisig_script.get_p2sh_p2wsh_address()\n",
    "print(f\"P2SH-P2WSH address: {p2sh_p2wsh_addr.to_string()}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Taproot Addresses with Script Paths\n",
    "\n",
    "Taproot allows defining alternative spending conditions through script paths."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate a key for the Taproot internal key\n",
    "internal_key = PrivateKey().get_public_key()\n",
    "\n",
    "# Create script paths\n",
    "# Example 1: Simple script path with a single public key\n",
    "script_pub_key = PrivateKey().get_public_key()\n",
    "script_path1 = Script([script_pub_key.to_hex(), 'OP_CHECKSIG'])\n",
    "\n",
    "# Example 2: Time-locked script path\n",
    "script_path2 = Script(['OP_DUP', 'OP_HASH160', script_pub_key.to_hash160(), 'OP_EQUALVERIFY', 'OP_CHECKSIG', 'OP_CHECKSEQUENCEVERIFY'])\n",
    "\n",
    "# Create a Taproot address with script paths\n",
    "p2tr_with_scripts_addr = internal_key.get_taproot_address([script_path1, script_path2])\n",
    "print(f\"Taproot address with script paths: {p2tr_with_scripts_addr.to_string()}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Signing and Verifying Messages\n",
    "\n",
    "Private and public keys can be used to sign and verify messages."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate a private key\n",
    "priv_key = PrivateKey()\n",
    "pub_key = priv_key.get_public_key()\n",
    "address = pub_key.get_address()\n",
    "\n",
    "# Sign a message\n",
    "message = \"Hello, Bitcoin!\"\n",
    "signature = priv_key.sign_message(message)\n",
    "print(f\"Message: {message}\")\n",
    "print(f\"Signature: {signature}\")\n",
    "print(f\"Address: {address.to_string()}\")\n",
    "\n",
    "# Verify the message\n",
    "is_valid = PublicKey.verify_message(address.to_string(), signature, message)\n",
    "print(f\"Signature valid: {is_valid}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Working with Address Objects Directly\n",
    "\n",
    "You can also create address objects directly from address strings or hash160 values."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create addresses from strings\n",
    "p2pkh_addr_str = \"mzF2sbdxcMqKFLoakdBcvZpUXMjgiXGZW1\"  # Example testnet address\n",
    "p2pkh_addr = P2pkhAddress.from_address(p2pkh_addr_str)\n",
    "print(f\"P2PKH address from string: {p2pkh_addr.to_string()}\")\n",
    "print(f\"P2PKH hash160: {p2pkh_addr.to_hash160()}\")\n",
    "\n",
    "# Create an address from a hash160 value\n",
    "hash160_value = p2pkh_addr.to_hash160()\n",
    "p2pkh_addr_from_hash = P2pkhAddress.from_hash160(hash160_value)\n",
    "print(f\"P2PKH address from hash160: {p2pkh_addr_from_hash.to_string()}\")\n",
    "\n",
    "# Get the scriptPubKey for an address\n",
    "script_pub_key = p2pkh_addr.to_script_pub_key()\n",
    "print(f\"ScriptPubKey: {script_pub_key.to_string()}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Converting Between Address Types\n",
    "\n",
    "When you have a private key, you can easily convert between different address types."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate a private key\n",
    "priv_key = PrivateKey()\n",
    "pub_key = priv_key.get_public_key()\n",
    "\n",
    "# Generate different address types from the same key\n",
    "addresses = {\n",
    "    \"P2PKH\": pub_key.get_address().to_string(),\n",
    "    \"P2WPKH\": pub_key.get_segwit_address().to_string(),\n",
    "    \"P2SH-P2WPKH\": pub_key.get_p2sh_p2wpkh_address().to_string(),\n",
    "    \"P2TR\": pub_key.get_taproot_address().to_string()\n",
    "}\n",
    "\n",
    "print(\"Different address types from the same key:\")\n",
    "for addr_type, addr in addresses.items():\n",
    "    print(f\"{addr_type}: {addr}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Conclusion\n",
    "\n",
    "This notebook demonstrated how to work with Bitcoin keys and addresses using the `bitcoinutils` library. We covered:\n",
    "\n",
    "1. Creating and managing private keys\n",
    "2. Deriving public keys\n",
    "3. Creating various types of addresses\n",
    "4. Working with script-based addresses\n",
    "5. Signing and verifying messages\n",
    "\n",
    "These building blocks are essential for creating and working with Bitcoin transactions."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}